# A0. Daily Chores

1. RotoWire Crosswalk
    Updates crosswalk of DraftKings names to RotoWire IDs
2. Projections - Daily Fantasy Fuel
    Clicks link to download projections
3. Projections - RotoWire
    Scrapes point and ownership projections
4. Odds - Fantasy Labs
    Scrapes game odds 
    
Note: 
1. The best time to run this is probably about an hour or so before start time so the projections are reasonably updated and the correct slate is chosen
2. You may want to check that projections are from the same slate

## IMPORTANT - RETRIEVE LAST NIGHT'S DK RESULTS AND PAYOUTS

### Imports

In [1]:
import requests
import datetime
import pandas as pd
from openpyxl import load_workbook
import os
import time
import numpy as np
from datetime import date


import sys
sys.path.append(r"C:\Users\james\Documents\MLB\Code")
from Utilities import *

# import import_ipynb
# from Utilities import *

import shutil

import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.simplefilter(action="ignore")

baseball_path = r"C:\Users\james\Documents\MLB\Data"
download_path = r"C:\Users\james\Downloads"

In [2]:
# Team Map
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))
# Full team name, Fantasypros abbreviation, Baseball Reference abbreviation, Rotowire abbreviation
team_map = team_map[['FULLNAME', 'FANPROSTEAM', 'BBREFTEAM', 'ROTOWIRETEAM']]

In [3]:
# Today's Date
# YYYY-MM-DD (datetime)
todaysdate_dt = datetime.date.today()

# YYYY-MM-DD (string)
todaysdate_dash = str(todaysdate_dt)

# MM/DD/YYYY
todaysdate_slash = todaysdate_dash.split("-")
todaysdate_slash = todaysdate_slash[1] + "/" + todaysdate_slash[2] + "/" + todaysdate_slash[0]

# YYYYMMDD
todaysdate = todaysdate_dash.replace("-", "")

In [4]:
print(todaysdate)

20230614


### RotoWire Crosswalk

In [5]:
# Read in crosswalk
crosswalk_df = pd.read_csv(os.path.join(baseball_path, "Utilities", "RotoWire Crosswalk.csv"))

# Makes lists from columns
name_list = crosswalk_df['name'].tolist()
team_list = crosswalk_df['team'].tolist()
name_plus_team_list = crosswalk_df['name_plus_team'].tolist()
rotowire_id_list = crosswalk_df['rotowire_id'].tolist()

In [6]:
# Read in Salaries
salaries = pd.read_csv(os.path.join(baseball_path, "A7. Matchups - 1. Salaries", "DKSalaries_" + todaysdate + ".csv"))

In [7]:
# Loop over names in Salaries, adding to lists if the player isn't in them already
for i in range(len(salaries)):  
    name = salaries['Name'][i]
    team = salaries['TeamAbbrev'][i]
    name_plus_team = name + " " + team
    
    if name_plus_team not in name_plus_team_list:
        
        rotowire_id = new_ids(salaries['Name'][i], salaries['TeamAbbrev'][i], "rotowire.com")
        
        name_list.append(name)
        team_list.append(team)
        name_plus_team_list.append(name_plus_team)
        rotowire_id_list.append(rotowire_id)

        print(name, ": ", rotowire_id)
        time.sleep(5)    

Ty Adcock :  16213


In [8]:
# Create a dictionary with column names as keys and lists as values
data = {'name': name_list,
        'team': team_list,
        'name_plus_team': name_plus_team_list,
        'rotowire_id': rotowire_id_list}

# Create a DataFrame from the dictionary
crosswalk_df = pd.DataFrame(data)

# Save CSV
crosswalk_df.to_csv(os.path.join(baseball_path, "Utilities", "RotoWire Crosswalk.csv"))

### Projections - Daily Fantasy Fuel

In [9]:
# Driver
driver = webdriver.Chrome(executable_path=r'C:\Users\james\Documents\MLB\chromedriver.exe')
driver.get('https://www.dailyfantasyfuel.com/mlb/projections/')

# Wait
wait = WebDriverWait(driver, 10)

# Click link
element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="projections"]/div[1]/div[2]/div[2]/div/span')))
element.click()

# Wait
time.sleep(3)

# Quit
driver.quit()

In [10]:
# Copy most recent download into the projections
# Get the list of files in the downloads folder
files = os.listdir(download_path)

# Get the most recently modified file
most_recent_file = max(files, key=lambda x: os.path.getctime(os.path.join(download_path, x)))

# Construct the full path to the most recent file
most_recent_file_path = os.path.join(download_path, most_recent_file)

# Move the most recent file to the destination folder
try:
    shutil.move(most_recent_file_path, os.path.join(baseball_path, "Projections - DFF"))
except:
    print("It's already there, silly!")

It's already there, silly!


In [11]:
df = pd.read_csv(os.path.join(baseball_path, "Projections - DFF", most_recent_file))
df

,first_name,last_name,position,injury_status,game_date,slate,team,opp,confirmed_order,starting_pitcher,...,over_under,implied_team_score,salary,L5_ppg_floor,L5_ppg_avg,L5_ppg_max,ppg_projection,value_projection,ppg_actual,value_actual
0,Tyler,Glasnow,P,NaN,2023-06-14,NaN,TB,OAK,NaN,YES,...,8.5,5.3,9400,15.6,19.5,25.1,23.2,2.47,NaN,NaN
1,Clayton,Kershaw,P,NaN,2023-06-14,NaN,LAD,CWS,NaN,YES,...,8.5,5.2,10800,6.4,18.3,33.6,22.6,2.09,NaN,NaN
2,Luis,Castillo,P,NaN,2023-06-14,NaN,SEA,MIA,NaN,YES,...,7.0,4.0,9800,8.5,22.2,35.7,21.1,2.15,NaN,NaN
3,Framber,Valdez,P,NaN,2023-06-14,NaN,HOU,WAS,NaN,YES,...,7.5,4.6,10300,10.5,22.9,40.9,19.1,1.85,NaN,NaN
4,Merrill,Kelly,P,NaN,2023-06-14,NaN,ARI,PHI,NaN,YES,...,8.5,4.5,9600,16.3,24.9,32.8,18.1,1.89,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,Tucupita,Marcano,SS,NaN,2023-06-14,NaN,PIT,CHC,NaN,NaN,...,9.0,4.0,2600,0.0,2.6,5.0,0.8,0.31,NaN,NaN
282,Gavin,Sheets,OF,NaN,2023-06-14,NaN,CWS,LAD,NaN,NaN,...,8.5,3.3,2300,0.0,1.2,3.0,0.7,0.30,NaN,NaN
283,Willie,MacIver,C,O,2023-06-14,NaN,COL,BOS,NaN,NaN,...,9.5,3.8,2000,NaN,NaN,NaN,0.0,0.00,NaN,NaN
284,Rylan,Bannon,2B,O,2023-06-14,NaN,HOU,WAS,NaN,NaN,...,7.5,4.6,2000,0.0,0.4,2.0,0.0,0.00,NaN,NaN


### Projections - RotoWire

In [12]:
# Create a webdriver instance
driver = webdriver.Chrome(executable_path=r'C:\Users\james\Documents\MLB\chromedriver.exe')

# Navigate to the website
driver.get('https://www.rotowire.com/baseball/daily-lineups.php')

# Get the page source
page_source = driver.page_source
    
# Regular expression pattern
pattern = r'is-selected" href="/baseball/daily-lineups.php\?slateID=\d{5}'

# Find the match
match = re.search(pattern, page_source)

# Extract the four digit number
if match:
    slateID = match.group(0)[-5:]
    print(slateID)
else:
    print('No match found')
    
driver.quit()


10918


In [13]:
# slateID = 10636
# todaysdate = "20230603"

In [14]:
def player_data(slateID):
    slateID = str(slateID)
    url = 'https://www.rotowire.com/daily/tables/optimizer-mlb.php'
    payload = {
    'siteID': '1',
    'slateID': slateID,
    'projSource': 'RotoWire',
    'rst': 'RotoWire'
    }
    jsonData = requests.get(url, params=payload).json()
    # jsonData = requests.get(url).json()
    df = pd.DataFrame(jsonData)

    df = df[['id', 'rotoPlayerID', 'player', 'ownership', 'proj_points', 'team', 'game_location']]
    
    
    return(df)

In [15]:
def matchup_data(slateID):
    url = f'https://www.rotowire.com/daily/tables/mlb/schedule.php?&siteID=1&slateID={slateID}'
    jsonData = requests.get(url).json()
    df = pd.DataFrame(jsonData)
    
    df = df[['date', 'visit_team', 'home_team']]

    
    return df

In [16]:
def scrape_rotowire(slateID):
    player_df = player_data(slateID)
    matchup_df = matchup_data(slateID)
    
    df = player_df.merge(matchup_df, left_on='team', right_on=['visit_team'], how='left', suffixes=("", "away"))
    df = df.merge(matchup_df, left_on='team', right_on=['home_team'], how='left', suffixes=("", "home"))
    
    df['date'].fillna(df['datehome'],inplace=True)
    df['visit_team'].fillna(df['visit_teamhome'],inplace=True)
    df['home_team'].fillna(df['home_teamhome'],inplace=True)
    
    #Merge with team map
    df = df.merge(team_map, left_on='visit_team', right_on='ROTOWIRETEAM', how='left',suffixes=("", "away"))
    df = df.merge(team_map, left_on='home_team', right_on='ROTOWIRETEAM', how='left',suffixes=("", "home"))
    
    df['Game Info'] = df['BBREFTEAM'] + "@" + df['BBREFTEAMhome']
                          
    df['day'] = df['date'].str.slice(0,3)
    df['num_matchups'] = len(matchup_df)
        
    df = df[['id', 'player', 'Game Info', 'date', 'day', 'proj_points', 'ownership', 'num_matchups']]
    
    df['proj_points'] = pd.to_numeric(df['proj_points'])
    df['ownership'] = pd.to_numeric(df['ownership'])
    
    df['has_ownership'] = ((df['ownership'].mean()) > 0)
    df['has_ownership'] = df['has_ownership'].astype('int')
    
    df['slateID'] = slateID
        
    filename = "Ownership_" + todaysdate + ".csv"
    df.to_csv(os.path.join(baseball_path, "Projections - RotoWire", filename))
        
    return df

df = scrape_rotowire(slateID)
df = df.sort_values('ownership', ascending=False)
df

,id,player,Game Info,date,day,proj_points,ownership,num_matchups,has_ownership,slateID
1,9276,Clayton Kershaw,CHW@LAD,WED: 10:10 PM,WED,21.76,21.72,12,1,10918
0,12248,Tyler Glasnow,TBR@OAK,WED: 9:40 PM,WED,23.28,19.94,12,1,10918
5,10510,Gerrit Cole,NYY@NYM,WED: 7:10 PM,WED,17.61,18.11,12,1,10918
2,14495,Luis Castillo,MIA@SEA,WED: 9:40 PM,WED,21.11,16.98,12,1,10918
3,14900,Framber Valdez,WSN@HOU,WED: 8:10 PM,WED,19.01,15.38,12,1,10918
...,...,...,...,...,...,...,...,...,...,...
492,17794,Wilyer Abreu,COL@BOS,WED: 7:10 PM,WED,0.00,0.00,12,1,10918
491,17594,Jack Larsen,MIA@SEA,WED: 9:40 PM,WED,0.00,0.00,12,1,10918
490,17796,Enmanuel Valdez,COL@BOS,WED: 7:10 PM,WED,0.00,0.00,12,1,10918
488,17809,Vinny Capra,PIT@CHC,WED: 8:05 PM,WED,0.00,0.00,12,1,10918


In [17]:
df['Game Info'].unique()

array(['CHW@LAD', 'TBR@OAK', 'NYY@NYM', 'MIA@SEA', 'WSN@HOU', 'COL@BOS',
       'LAA@TEX', 'PHI@ARI', 'CLE@SDP', 'TOR@BAL', 'CIN@KCR', 'PIT@CHC'],
      dtype=object)

### Odds - Fantasy Labs

In [18]:
# Has a lot more fun data too! Including, weirdly, team colors! Might want to add more to this later
def gamble_df(jsonData):
    list_list = []
    for i in range(len(jsonData)):
        try:
            home_name = jsonData[i]['EventDetails']['Properties']['HomeTeamShort']
            away_name = jsonData[i]['EventDetails']['Properties']['VisitorTeamShort']
            home_runs = jsonData[i]['EventDetails']['Properties']['HomeVegasRuns']
            away_runs = jsonData[i]['EventDetails']['Properties']['VisitorVegasRuns']

            away_ml = jsonData[i]['EventDetails']['Properties']['VisitorGameMoneylineCurrent']
            home_ml = jsonData[i]['EventDetails']['Properties']['HomeGameMoneylineCurrent']

            home_ou = jsonData[i]['EventDetails']['Properties']['HomeGameOUCurrent']
            home_ou_juice = jsonData[i]['EventDetails']['Properties']['HomeGameOUJuiceCurrent']

            home_spread = jsonData[i]['EventDetails']['Properties']['HomeGameSpreadCurrent']
            home_spread_juice = jsonData[i]['EventDetails']['Properties']['HomeGameSpreadJuiceCurrent']

            date = jsonData[i]['EventDetails']['Properties']['EventDateId']

            gamble_list = [away_name, away_runs, home_name, home_runs, home_ou, home_ou_juice, home_spread, home_spread_juice, away_ml, home_ml, date]
            list_list.append(gamble_list)
        except:
            print("Broke")
    
    df = pd.DataFrame(list_list, columns=['away_team','away_runs', 'home_team', 'home_runs', 'ou', 'ou_juice', 'spread', 'spread_juice', 'away_ml', 'home_ml', 'date'])
    
    return df

In [19]:
def daily_gamble(date):
    print(date)
    dateStr = date[0:4] + "-" + date[4:6] + "-" + date[6:8]
    
    url = f'https://www.fantasylabs.com/api/sportevents/3/{dateStr}/vegas/?sportId=3&date={dateStr}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    jsonData = requests.get(url, headers=headers).json()
    
    # try:
    df = gamble_df(jsonData)
    
    
    df = df.merge(team_map, left_on='away_team', right_on='FANPROSTEAM', how='left')
    df['away_team'] = df['BBREFTEAM']
    df.drop(columns={'FULLNAME', 'FANPROSTEAM', 'BBREFTEAM'}, inplace=True)
    
    df = df.merge(team_map, left_on='home_team', right_on='FANPROSTEAM', how='left')
    df['home_team'] = df['BBREFTEAM']
    df.drop(columns={'FULLNAME', 'FANPROSTEAM', 'BBREFTEAM'}, inplace=True)

    odds_filename = "Odds " + date + ".csv"
    df.to_csv(os.path.join(baseball_path, "Odds", odds_filename))
    
daily_gamble(todaysdate)

20230614


In [20]:
print("Code was last run on: {} at {}.".format(datetime.date.today(), datetime.datetime.now().strftime("%H:%M:%S")))

Code was last run on: 2023-06-14 at 18:20:06.


# To do list
### Short-term
Projection accuracy by sample size <br>

Audits: <br>
Pulls <br>
Simulation <br>

### Medium-term
Audits: <br>
Advance/DP/DP base rates <br>
Closers - why are they unnecessary? <br>

inning dummy? <br>
times through <br>

A file to run before optimizer with: <br>
Evaluate Scores (gamble picks) <br>
Evaluate Day (player oddities) <br>
Weather warnings <br>
Clean weather <br>


### Long-term
Dashboard <br>
Other stats: <br>
Batters faced <br>
Automate DK (if possible) <br>
